<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myProspects_V20_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance

In [2]:
import pandas as pd
import numpy as np

mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Tradevesting/myProspectsScrips.csv')

#mypf = mypf[mypf['Nifty100'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC','EQUITAS', 'SCHAEFFLER ', 'ADANITRANS']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

stock_ids = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

stock_ids.sort()
#stock_ids

In [3]:
# import libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()

In [4]:
import yfinance as yfin
def get_stock_v20(stock_symbol):
    #print(stock_symbol)
    stock_df = yfin.Ticker(stock_symbol).history(period='3y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Candle'] = 'R'
    stock_df.loc[stock_df['Close'] > stock_df['Open'], 'Candle'] = 'G'
    stock_df['Prev_Candle'] = stock_df['Candle'].shift(1)
    stock_df['Next_Candle'] = stock_df['Candle'].shift(-1)
    stock_df['Signal'] = 'NA'
    stock_df.loc[(stock_df['Candle'] != stock_df['Prev_Candle']) & (stock_df['Candle'] == 'G'), 'Signal'] = 'Start'
    stock_df.loc[(stock_df['Candle'] != stock_df['Next_Candle']) & (stock_df['Next_Candle'] == 'R') & (stock_df['Signal'] == 'NA'), 'Signal'] = 'End'
    stock_df.loc[(stock_df['Candle'] != stock_df['Next_Candle']) & (stock_df['Next_Candle'] == 'R') & (stock_df['Signal'] == 'Start'), 'Signal'] = 'Start/End'
    stock_df = stock_df[stock_df['Signal'] != 'NA']
    stock_df['Prev_Low'] = stock_df['Low'].shift(1)
    stock_df = stock_df[stock_df['Signal'] != 'Start']
    stock_df.loc[(stock_df['Signal'] == 'End'), 'Low'] = stock_df['Prev_Low']
    stock_df['Up%'] = round((stock_df['High'] - stock_df['Low'])*100/stock_df['Low'],2)
    stock_df = stock_df[stock_df['Up%'] > 20]
    stock_df['Current'] = yfin.Ticker(stock_symbol).history(period='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]['Close'].values[0]
    stock_df = stock_df[stock_df['Current'] < stock_df['Low']]
    stock_df['Stock'] = stock_symbol
    return stock_df

def get_v20():
    df_v20 = pd.DataFrame()
    for stock_id in stock_ids:
        #print(stock_id)
        df_tmp = get_stock_v20(stock_id+'.NS')
        df_tmp['Criteria'] = mypf[(mypf['Symbol']) == stock_id.upper()]['Criteria'].values[0]
        df_v20 = df_v20.append(df_tmp)

    return df_v20.sort_index()

In [5]:
cols = ['Stock', 'Up%', '200_SMA', 'Close' ,'Criteria']
df_v20_prospects = get_v20()[cols]

In [6]:
df_v20_prospects_200SMA = df_v20_prospects[(df_v20_prospects['Close'] <= df_v20_prospects['200_SMA'])]

In [7]:
df_v20_prospects_V40N = df_v20_prospects[(df_v20_prospects['Criteria'] == 'V40N') | (df_v20_prospects['Criteria'] == 'V40')]

In [8]:
pd.concat([df_v20_prospects_V40N, df_v20_prospects_200SMA])

,Stock,Up%,200_SMA,Close,Criteria
Date,,,,,
2020-12-28 00:00:00+05:30,WHIRLPOOL.NS,24.03,2129.0,2558.207764,V40
2020-12-28 00:00:00+05:30,JCHAC.NS,30.12,2203.0,2635.550049,V40N
2021-02-02 00:00:00+05:30,INDIGOPNTS.NS,28.45,3104.0,3104.490479,V40N
2021-02-09 00:00:00+05:30,TEAMLEASE.NS,38.48,2606.0,3577.149902,V40N
2021-03-02 00:00:00+05:30,SYMPHONY.NS,40.43,943.0,1416.934570,V40N
2021-04-06 00:00:00+05:30,LALPATHLAB.NS,24.44,2301.0,3017.052979,V40N
2021-05-28 00:00:00+05:30,LUXIND.NS,61.36,1725.0,3233.427734,V40N
2021-10-27 00:00:00+05:30,TTKPRESTIG.NS,23.52,795.0,1097.593140,V40N
2021-11-18 00:00:00+05:30,SFL.NS,20.50,1122.0,1593.900024,V40N


In [9]:
df_v20_prospects['Criteria'].value_counts()

VR      34
V40N    13
AR      10
V40      1
Name: Criteria, dtype: int64